# 네이버 항공권 웹스크래핑

설정부분

In [17]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
import random
from datetime import datetime

# 크롬 옵션 설정 - 셀레니움 접근 제한 : 보안
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 티 안 나게
options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

In [18]:
# 브라우저 실행
browser = webdriver.Chrome(options=options) 
browser.maximize_window() # 창 최대화

In [19]:
# 1. 네이버 항공권 접속
url = "https://flight.naver.com/"
browser.get(url)

In [20]:
# 팝업창 닫기
time.sleep(2)
elem = browser.find_element(By.CLASS_NAME,"FullscreenPopup_close")
elem.click()

In [21]:
# 3.출발지 선택
elem = browser.find_element(By.CLASS_NAME,"select_code__IVa3P").click()

In [22]:
time.sleep(2)
# 4. 김포 선택
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/div/div/ul[1]/li[3]/button')
elem.click()
# 5. 도착지 선택
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[1]/button[2]/i')
elem.click()
time.sleep(2)
# 6. 제주 선택
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/div[2]/div[2]/ul[1]/li[1]/button')
elem.click()

In [23]:
# 7. 가는날 선택
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[2]/button[1]')
elem.click()
# 8. 출발날짜 선택
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/div[1]/div[2]/div/div/div[2]/table/tbody/tr[5]/td[7]/button/b')
elem.click()

# 9. 도착날짜 선택 
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/div[1]/div[2]/div/div/div[3]/table/tbody/tr[1]/td[4]/button/b')
elem.click()
# 10. 항공권 검색
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/button')
elem.click()

In [24]:
### 스크롤 내리기
# 현재 사이트 높이를 가져오는 자바스크립트
prev_height = browser.execute_script("return document.body.scrollHeight")

# 반복 실행
while True:
    # 현재 브라우저에서 0에서 현재 높이까지 스크롤 내리기
    # 자바 스크립트 실행 
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(2)
    # 변동된 현재 문서 높이를 가져오기
    curr_height = browser.execute_script("return document.body.scrollHeight")
    # 스크롤 높이가 변동이 없을 시 
    if curr_height == prev_height: break 
    prev_height = curr_height # 현재 높이를 다시 입력해서 스크롤 내리기 재실행
    
print("스크롤 내리기 종료")

스크롤 내리기 종료


##### 네이버 항공권
##### 김포, 제주 5/31 6/1 ->
##### 금액 90000원 이하는 제외
##### 출발 시간 17:00 이상은 제외 


In [25]:
soup = BeautifulSoup(browser.page_source,"lxml")

# 비행정보 모두 가져오기 
datas = soup.find_all("div",{"class":"domestic_Flight__8bR_b"})
print(len(datas)) # 287
datas_list = [] # list를 정렬할때 사용
for data in datas:
    # 출발시간
    times = data.find_all("b",{"class":"route_time__xWu7a"})
    startTime = times[0].get_text().strip()
    print(startTime)
    # 항공사 이름
    airline = data.find("b",{"class":"airline_name__0Tw5w"}).get_text().strip()
    print(airline)
    # 도착시간
    endTime = times[1].get_text().strip()
    print(endTime)
    # 가격 천단위 제거 
    price = data.find("i",{"class":"domestic_num__ShOub"}).get_text().strip().replace(",","")
    price = int(price)
    print(price)
    
    print('-'*50)
    
    datas_list.append([airline,startTime,endTime,price])

132
06:05
이스타항공
07:20
107400
--------------------------------------------------
06:05
이스타항공
07:20
118600
--------------------------------------------------
06:10
이스타항공
07:25
107400
--------------------------------------------------
06:10
이스타항공
07:25
118600
--------------------------------------------------
06:10
아시아나항공
07:25
125700
--------------------------------------------------
06:10
아시아나항공
07:25
131700
--------------------------------------------------
06:15
진에어
07:30
101600
--------------------------------------------------
06:15
제주항공
07:30
103600
--------------------------------------------------
06:15
제주항공
07:30
118200
--------------------------------------------------
06:15
진에어
07:30
118700
--------------------------------------------------
06:20
제주항공
07:35
103600
--------------------------------------------------
06:20
제주항공
07:35
118200
--------------------------------------------------
06:25
티웨이항공
07:40
102700
--------------------------------------------------
06:25
티웨이항공
07